In [8]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import cumfreq
from scipy.spatial import distance
from scipy.spatial.distance import euclidean
import csv
import os

dir_base = '/content/drive/My Drive/SBRC 2024/results_sbrc_models_fl-cfa/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Inicialize as variáveis para a geração dos gráficos
num_users = 10

In [10]:
# Função para calcular a distância euclidiana entre dois pontos
def calculate_euclidean_distance(point1, point2):
    return distance.euclidean(point1[:2], point2[:2])

def get_cumulate_error(user_id, model_name):
  location_errors = []
  orientation_errors = []

  # Nome do arquivo CSV para o usuário atual
  file_name = f"{user_id}_{model_name.lower()}_model_tracking_data.csv"
  file_path = os.path.join(dir_base, file_name)

  # Inicialize os arrays para os dados do usuário atual
  user_real_locations      = []
  user_predicted_locations = []
  user_orientation_errors  = []

  # Carregue os dados do arquivo CSV
  with open(file_path, 'r') as file:
      reader = csv.reader(file)

      # Leitura dos dados do arquivo CSV
      for row in reader:
          real_data = np.array([float(coord) for coord in row[1].strip('[]').split()]) # Converte a string para array NumPy
          predicted_data = np.array([float(coord) for coord in row[2].strip('[]').split()])

          user_real_locations.append(real_data)
          user_predicted_locations.append(predicted_data)

          # Cálculo dos erros de orientação
          user_orientation_errors.append(np.abs(predicted_data[2] - real_data[2]))

  user_real_locations = np.array(user_real_locations)
  user_predicted_locations = np.array(user_predicted_locations)

  # Cálculo dos erros de localização (distância euclidiana)
  user_location_errors = np.array([calculate_euclidean_distance(real, predicted) for real, predicted in zip(user_real_locations, user_predicted_locations)])

  user_orientation_errors = np.array(user_orientation_errors)

  return [user_location_errors, user_orientation_errors]

In [11]:
def organize_data():
    erros_por_modelo = {'ESN': [], 'GRU': [], 'LSTM': []}

    for modelo in erros_por_modelo:
        for user_id in range(0,10):
            erros_por_modelo[modelo].append(get_cumulate_error(user_id, modelo))


    errors_loc = []
    errors_ori = []
    for i in range(10):
        errors_loc.append([erro_localizacao for modelo_erros in erros_por_modelo.values() for erro_localizacao, _ in modelo_erros[i:i+1]])
        errors_ori.append([erro_orientacao for modelo_erros in erros_por_modelo.values() for _, erro_orientacao in modelo_erros[i:i+1]])

    return errors_loc, errors_ori

In [12]:
def plot_localization(errors_loc):
    fig, axs = plt.subplots(figsize=(24, 6))

    index = 0
    user = 1
    for item in errors_loc:
        boxp = axs.boxplot(item, showfliers=False, vert=True, positions=[index, index + 0.2, index + 0.4], patch_artist=True)


        ind = 0
        for box in (boxp['boxes']):
            if ind == 0:
                box.set(facecolor = '#33adff')
                box.set_edgecolor(color='black')

            if ind == 1:
                box.set(facecolor = '#e6b800' )
                box.set_edgecolor(color='black')

            if ind == 2:
                box.set(facecolor = '#cc0066' )
                box.set_edgecolor(color='black')

            ind = ind + 1

        user = user + 1

        index = index + 0.8

    axs.legend([boxp["boxes"][0], boxp["boxes"][1], boxp["boxes"][2]], ['ESN', 'GRU', 'LSTM'], loc='upper right', fontsize=22)
    axs.set_xticklabels(labels=['', '', '']*10, fontsize=22)
    axs.set_ylabel('Localização (Metros)', fontsize=22)
    axs.set_yticks([0, 40, 80, 120])
    axs.set_yticklabels([0, 40, 80, 120], fontsize=22)
    axs.grid(axis='y')

    #plt.yscale("log")

    plt.savefig("boxplot_localization.pdf", bbox_inches='tight')

In [13]:
def plot_orientation(errors_ori):
    fig, axs = plt.subplots(figsize=(24, 6))

    index = 0
    user = 1
    for item in errors_ori:
        boxp = axs.boxplot(item, showfliers=False, vert=True, positions=[index, index + 0.2, index + 0.4], patch_artist=True)

        ind = 0
        for box in (boxp['boxes']):
            if ind == 0:
                box.set(facecolor = '#33adff')
                box.set_edgecolor(color='black')

            if ind == 1:
                box.set(facecolor = '#e6b800' )
                box.set_edgecolor(color='black')

            if ind == 2:
                box.set(facecolor = '#cc0066' )
                box.set_edgecolor(color='black')

            ind = ind + 1

        user = user + 1

        index = index + 0.8

    l = []
    for i in range(1,11):
        l = l + ['', 'usuário ' + str(i), '']


    axs.legend([boxp["boxes"][0], boxp["boxes"][1], boxp["boxes"][2]], ['ESN', 'GRU', 'LSTM'], loc='upper right', fontsize=22)
    axs.set_xticklabels(labels=l, fontsize=22)
    axs.set_ylabel('Orientação (Graus)', fontsize=22)
    axs.set_yticks([0, 200, 400, 600])
    axs.set_yticklabels([0, 200, 400, 600], fontsize=22)
    axs.grid(axis='y')

    #plt.yscale("log")

    plt.savefig("boxplot_orientation.pdf", bbox_inches='tight')

In [ ]:
errors_loc, errors_ori = organize_data()

plot_localization(errors_loc)
plot_orientation(errors_ori)